# Cell recruitment impediment

In [1]:
# required libraries
import numpy as np
import os,sys
import pandas as pd
import time
import random

In [2]:
long_cell_cycle_length_mean = 340
long_cell_cycle_length_std = 32

In [3]:
def cellDivisions(tm,td,ts,stem,positions):
    
    # A cell divides when it completes its cell cycle,
    # meaning that the time remaining to cell division ('tr') reachs zero.
    
    global n # current number of cell in the tissue
    
    for dividingCell in range(0,n):
    # for each cell in the tissue        
        tc = ts - tm[dividingCell] # 'tc' is elapsed time since last division ('tm'), while 'ts' is the simulation time
        tr = td[dividingCell] - tc # 'tr' is the remaining time for cell division, while 'td' is the cell cycle length
        if tr <= 0:
        # if the cell completes its cell cycle, duplicates all cell properties array            
            tm[dividingCell] = ts
            tm = np.insert(tm,dividingCell,ts)
            tm = tm[:-1]            
            n = n+1 # increase the current number of cells in the tissue by 1
            positions = np.insert(positions,dividingCell,-1)
            td[dividingCell] = np.random.normal(long_cell_cycle_length_mean, long_cell_cycle_length_std)
            td = np.insert(td,dividingCell,np.random.normal(long_cell_cycle_length_mean, long_cell_cycle_length_std))
            td = td[:-1]               
    return tm,td,stem,positions

In [4]:
def run(steps,tm,td,stem,positions):
    
    # calls 'cellDivisions' each step and makes time goes by
    
    global n
    outgrowth = []
    tracking = []
    tracking.append(positions)
    for ts in range(0,steps+1,1):
        tm,td,stem,positions = cellDivisions(tm,td,ts,stem,positions)
        outgrowth.append(n-n0)    # Appends the outgrowth for each time
        tracking.append(positions)
    return outgrowth,tracking

In [5]:
n0 = 200  # Number of cells after amputation

root = 'simulations/'
model = 'impediment/'    
folder = 'n0='+str(n0)+'/'
if not os.path.isdir(root+model+folder):
    os.makedirs(root+model+folder)
    os.makedirs(root+model+folder+'outgrowth/')    
    
np.random.seed(1)

emptySpace = 250
lattice = n0+emptySpace
steps = 24*8  # Number of steaps (in hours)

print('n0:',n0)
print('-------')
for seed in range(0,1000):
# for each seed reset the number of cells, initialize cell properties, calls run and saves output    
    n = n0
    positions = np.arange(0,n0) 
    tm = np.random.uniform(-(long_cell_cycle_length_mean-2*long_cell_cycle_length_std),0,lattice)
    td = np.random.normal(long_cell_cycle_length_mean,long_cell_cycle_length_std,lattice)
    stem = np.zeros(lattice)
    solution,track = run(steps,tm,td,stem,positions)

    d = {"time":np.arange(0,steps+1),"outgrowth":solution}
    df = pd.DataFrame(d)
    outfile = open(root+model+folder+'outgrowth'+'/'+'sim_seed='+str(seed)+'.csv', 'a')
    df.to_csv(outfile, sep=',')
    outfile.close()

n0: 200
-------
